# 02. Feature Engineering – 파생변수 생성 (예시)
## 2-1. 라이브러리 & 데이터 로드
## 2-2. 날짜/시간 파생변수
## 2-3. 시계열 전용 피처
## 2-4. 외부데이터 병합(선택)
## 2-5. 최종 피처 선택 & 저장

In [ ]:
import pandas as pd
from src.data_loader import load_csv, save_processed
from src.features import add_datetime_features, add_lag_features, add_rolling_mean

# 1. 데이터 로드 (EDA에서 사용한 전처리된 버전 or raw)
df = load_csv("data/interim/01.bike/bike_clean.csv")  # 경로/파일명 각자 수정
df["datetime"] = pd.to_datetime(df["datetime"])
df = df.sort_values("datetime").reset_index(drop=True)

target_col = "demand"  # 각자 타겟 이름

In [ ]:
# 2. 날짜/시간 파생변수
df = add_datetime_features(df, "datetime")
df.head()

In [ ]:
# 3. lag / rolling feature 생성
df = add_lag_features(df, target_col, lags=[1, 24])       # 바로 이전/하루 전
df = add_rolling_mean(df, target_col, windows=[3, 7, 24]) # 3,7,24기간 이동평균

# lag/rolling 때문에 생긴 NaN 제거
df = df.dropna().reset_index(drop=True)

In [ ]:
# 4. (선택) 외부 데이터 병합 – 예시 틀
# weather = load_csv("data/interim/weather_clean.csv")
# df = df.merge(weather, on="datetime", how="left")
# ...

In [ ]:
# 5. 학습용 feature 목록 정리 & 저장
feature_cols = [
    "year", "month", "dayofweek", "hour",
    f"{target_col}_lag1", f"{target_col}_lag24",
    f"{target_col}_roll3", f"{target_col}_roll7", f"{target_col}_roll24",
    # 여기에 날씨/공사/카테고리 one-hot 등 추가
]

X = df[feature_cols]
y = df[target_col]

print(X.head())
print(y.head())

# 저장 (X, y 통째로 저장하거나 df 전체 저장 둘 다 가능)
save_processed(df, "bike_features.csv")

## 피처 요약 메모

- 사용한 피처:
  - 날짜/시간: year, month, dayofweek, hour
  - 시계열: lag1, lag24, roll3, roll7, roll24
  - 외부 변수: (예) 기온, 강수량, 공사여부 등

- 기대 효과:
  - (예) 시계열 패턴과 계절성, 날씨 영향을 함께 반영